# The ROCKET transform

## Overview

ROCKET [1] transforms time series using random convolutional kernels (random length, weights, bias, dilation, and padding). ROCKET computes two features from the resulting feature maps: the max, and the proportion of positive values (or ppv). The transformed features are used to train a linear classifier.

[1] Dempster A, Petitjean F, Webb GI (2019) ROCKET: Exceptionally fast and accurate time series classification using random convolutional kernels. [arXiv:1910.13051](https://arxiv.org/abs/1910.13051)

***

## Contents

1. Imports
2. Univariate Time Series
3. Multivariate Time Series
4. Pipeline Example

***

## 1 Imports

Import example data, ROCKET, and a classifier (`RidgeClassifierCV` from scikit-learn), as well as NumPy and `make_pipeline` from scikit-learn.

**Note**: ROCKET compiles (via Numba) on import, which may take a few seconds.

In [47]:
# !pip install --upgrade numba

In [48]:
import numpy as np
from sklearn.linear_model import RidgeClassifierCV
from sklearn.pipeline import make_pipeline

from aeon.datasets import load_basic_motions  # multivariate dataset
from aeon.datasets import load_gunpoint  # univariate dataset
from aeon.transformations.collection.convolution_based import Rocket

## 2 Univariate Time Series

We can transform the data using ROCKET and separately fit a classifier, or we can use ROCKET together with a classifier in a pipeline (section 4, below).

### 2.1 Load the Training Data
For more details on the data set, see the [univariate time series classification
notebook](https://github.com/aeon-toolkit/aeon/tree/main/examples/classification/classification.ipynb).

In [68]:
X_train, y_train = load_gunpoint(split="train")
X_train = X_train[:5, :, :]
y_train = y_train[:5]
print(X_train.shape)

(5, 1, 150)


In [72]:
X_train

array([[[-0.6478854 , -0.64199155, -0.63818632, -0.63825875,
         -0.63834515, -0.63869741, -0.64304876, -0.64376789,
         -0.64504991, -0.64711823, -0.64915334, -0.65124584,
         -0.65729046, -0.66220082, -0.66123227, -0.66098661,
         -0.66156196, -0.66225552, -0.66191228, -0.66274039,
         -0.66093438, -0.66344985, -0.66219448, -0.6623359 ,
         -0.66171176, -0.66139185, -0.66140829, -0.66144719,
         -0.66037322, -0.65910828, -0.65974177, -0.65817648,
         -0.65839527, -0.65734763, -0.65851971, -0.6584864 ,
         -0.657366  , -0.65435815, -0.65251819, -0.6513159 ,
         -0.64814481, -0.64733294, -0.64709792, -0.64486153,
         -0.64360412, -0.64349171, -0.64265061, -0.6420974 ,
         -0.64447929, -0.6488921 , -0.65765851, -0.66120325,
         -0.64966432, -0.61016844, -0.53743483, -0.46503123,
         -0.35853483, -0.28313648, -0.17744604,  0.15824648,
          0.35028455,  0.48241039,  0.60380734,  0.79710411,
          0.98755132,  1

In [86]:
from aeon.utils.conversion import convert_collection
from aeon.utils.validation import get_type
X=convert_collection(np.zeros(shape=(10, 3, 20)), "np-list")
type(X)




list

In [89]:
X.shape()

AttributeError: 'list' object has no attribute 'shape'

In [82]:
from aeon.utils.data_types import COLLECTIONS_DATA_TYPES

In [85]:
COLLECTIONS_DATA_TYPES

['numpy3D', 'np-list', 'df-list', 'numpy2D', 'pd-wide', 'pd-multiindex']

In [78]:
test = np.zeros(shape=(10, 3, 20))

In [79]:
get_type(test)

'numpy3D'

In [73]:
from aeon.utils.conversion import resolve_equal_length_inner_type
resolve_equal_length_inner_type(X_train)

ValueError: Error, no valid inner types in [[[-0.6478854  -0.64199155 -0.63818632 -0.63825875 -0.63834515
   -0.63869741 -0.64304876 -0.64376789 -0.64504991 -0.64711823
   -0.64915334 -0.65124584 -0.65729046 -0.66220082 -0.66123227
   -0.66098661 -0.66156196 -0.66225552 -0.66191228 -0.66274039
   -0.66093438 -0.66344985 -0.66219448 -0.6623359  -0.66171176
   -0.66139185 -0.66140829 -0.66144719 -0.66037322 -0.65910828
   -0.65974177 -0.65817648 -0.65839527 -0.65734763 -0.65851971
   -0.6584864  -0.657366   -0.65435815 -0.65251819 -0.6513159
   -0.64814481 -0.64733294 -0.64709792 -0.64486153 -0.64360412
   -0.64349171 -0.64265061 -0.6420974  -0.64447929 -0.6488921
   -0.65765851 -0.66120325 -0.64966432 -0.61016844 -0.53743483
   -0.46503123 -0.35853483 -0.28313648 -0.17744604  0.15824648
    0.35028455  0.48241039  0.60380734  0.79710411  0.98755132
    1.2037853   1.4065415   1.5962459   1.7128307   1.7788152
    1.7938492   1.7971408   1.8101928   1.8206277   1.8172235
    1.8316333   

### 2.2 Initialise ROCKET and Transform the Training Data

In [50]:
rocket = Rocket(n_kernels=10000)  # by default, ROCKET uses 10,000 kernels
rocket.fit(X_train)
X_train_transform = rocket.transform(X_train)
print(X_train_transform.shape)

(5, 20000)


### 2.3 Fit a Classifier

We recommend using `RidgeClassifierCV` from scikit-learn for smaller datasets (fewer than approx. 20K training examples), and using logistic regression trained using stochastic gradient descent for larger datasets.

In [51]:
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
classifier.fit(X_train_transform, y_train)

RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]))

### 2.4 Load and Transform the Test Data

In [52]:
X_test, y_test = load_gunpoint(split="test")
X_test_transform = rocket.transform(X_test)

### 2.5 Classify the Test Data

In [53]:
classifier.score(X_test_transform, y_test)

0.6133333333333333

***

## 3 Multivariate Time Series

We can use ROCKET in exactly the same way for multivariate time series.

### 3.1 Load the Training Data

In [65]:
X_train, y_train = load_basic_motions(split="train")

In [67]:
X_train.shape

(40, 6, 100)

### 3.2 Initialise ROCKET and Transform the Training Data

In [55]:
rocket = Rocket(n_kernels=100)  # by default, ROCKET uses 10,000 kernels
rocket.fit(X_train)
X_train_transform = rocket.transform(X_train)
X_train_transform.shape

(40, 200)

### 3.3 Fit a Classifier

In [56]:
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
classifier.fit(X_train_transform, y_train)

RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]))

### 3.4 Load and Transform the Test Data

In [57]:
X_test, y_test = load_basic_motions(split="test")
X_test_transform = rocket.transform(X_test)

### 3.5 Classify the Test Data

In [58]:
classifier.score(X_test_transform, y_test)

0.925

***

## 4 Pipeline Example

We can use ROCKET together with `RidgeClassifierCV` (or another classifier) in a pipeline.  We can then use the pipeline like a self-contained classifier, with a single call to `fit`, and without having to separately transform the data, etc.

### 4.1 Initialise the Pipeline

In [59]:
rocket_pipeline = make_pipeline(
    Rocket(), RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
)

### 4.2 Load and Fit the Training Data

In [60]:
# it is necessary to pass y_train to the pipeline
# y_train is not used for the transform, but it is used by the classifier
rocket_pipeline.fit(X_train, y_train)

Pipeline(steps=[('rocket', Rocket()),
                ('ridgeclassifiercv',
                 RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03])))])

### 4.3 Load and Classify the Test Data

In [61]:
rocket_pipeline.score(X_test, y_test)

0.975